<a href="https://colab.research.google.com/github/dlagur/ProjectHanium/blob/main/WebCrawlingByUsingKeyword.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
!pip install selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:7 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:9 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/

In [13]:
#Colab에선 웹브라우저 창이 뜨지 않으므로 별도 설정한다.
 
options = webdriver.ChromeOptions()
options.add_argument('--headless')        # Head-less 설정
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver', options=options)

In [14]:
#크롤링시 필요한 라이브러리 불러오기
from selenium import webdriver
from urllib.request import urlopen
from bs4 import BeautifulSoup
from urllib.parse import quote_plus
from selenium.webdriver.common.keys import Keys
import time

In [15]:
# 페이지 url 형식에 맞게 바꾸어 주는 함수 만들기
  #입력된 수를 1, 11, 21, 31 ...만들어 주는 함수
def makePgNum(num):
    if num == 1:
        return num
    elif num == 0:
        return num+1
    else:
        return num+9*(num-1)


# 크롤링할 url 생성하는 함수 만들기(검색어, 크롤링 시작 페이지, 크롤링 종료 페이지)
def makeUrl(search,start_pg,end_pg):
    if start_pg == end_pg:
        start_page = makePgNum(start_pg)
        url = "https://search.naver.com/search.naver?where=news&sm=tab_pge&query=" + search + "&start=" + str(start_page)
        print("생성url: ",url)
        return url
    else:
        urls= []
        for i in range(start_pg,end_pg+1):
            page = makePgNum(i)
            url = "https://search.naver.com/search.naver?where=news&sm=tab_pge&query=" + search + "&start=" + str(page)
            urls.append(url)
        print("생성url: ",urls)
        return urls

In [20]:
##########뉴스크롤링 시작###################

#검색어 입력
search = input("검색할 키워드를 입력해주세요:")

#검색 시작할 페이지 입력
page = int(input("\n크롤링할 시작 페이지를 입력해주세요. ex)1(숫자만입력):")) # ex)1 =1페이지,2=2페이지...
print("\n크롤링할 시작 페이지: ",page,"페이지")   
#검색 종료할 페이지 입력
page2 = int(input("\n크롤링할 종료 페이지를 입력해주세요. ex)1(숫자만입력):")) # ex)1 =1페이지,2=2페이지...
print("\n크롤링할 종료 페이지: ",page2,"페이지")   

# naver url 생성
search_urls = makeUrl(search,page,page2)

검색할 키워드를 입력해주세요:리더십

크롤링할 시작 페이지를 입력해주세요. ex)1(숫자만입력):1

크롤링할 시작 페이지:  1 페이지

크롤링할 종료 페이지를 입력해주세요. ex)1(숫자만입력):20

크롤링할 종료 페이지:  20 페이지
생성url:  ['https://search.naver.com/search.naver?where=news&sm=tab_pge&query=리더십&start=1', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=리더십&start=11', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=리더십&start=21', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=리더십&start=31', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=리더십&start=41', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=리더십&start=51', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=리더십&start=61', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=리더십&start=71', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=리더십&start=81', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=리더십&start=91', 'https://search.naver.com/search.nav

In [25]:
## selenium으로 navernews만 뽑아오기##
# 버전에 상관 없이 os에 설치된 크롬 브라우저 사용
driver = webdriver.Chrome('chromedriver', options=options)
driver.implicitly_wait(3)


# selenium으로 검색 페이지 불러오기 #

naver_urls=[]

for i in search_urls:
    driver.get(i)
    time.sleep(1) #대기시간 변경 가능

    # 네이버 기사 눌러서 제목 및 본문 가져오기#
    # 네이버 기사가 있는 기사 css selector 모아오기
    a = driver.find_elements(By.CSS_SELECTOR,'a.info')

    # 위에서 생성한 css selector list 하나씩 클릭하여 본문 url얻기
    for i in a:
        i.click()

        # 현재탭에 접근
        driver.switch_to.window(driver.window_handles[1])
        time.sleep(3) #대기시간 변경 가능

        # 네이버 뉴스 url만 가져오기

        url = driver.current_url
        print(url)

        if "news.naver.com" in url:
            naver_urls.append(url)
  
        else:
            pass
        
        # 현재 탭 닫기
        driver.close()

        # 다시처음 탭으로 돌아가기(매우 중요!!!)
        driver.switch_to.window(driver.window_handles[0])

print(naver_urls)

https://www.hankyung.com/
https://n.news.naver.com/mnews/article/015/0004720789?sid=101
https://www.ytn.co.kr/
https://n.news.naver.com/mnews/article/052/0001760486?sid=100
http://www.chungnamilbo.co.kr/
https://newsis.com/
https://newsis.com/ad/ad_20220627.html
https://n.news.naver.com/mnews/article/003/0011288335?sid=100
https://www.news1.kr/
https://n.news.naver.com/mnews/article/421/0006198312?sid=102
http://www.sisajournal.com/
https://n.news.naver.com/mnews/article/586/0000041211?sid=100
https://imnews.imbc.com/pc_main.html
https://n.news.naver.com/mnews/article/214/0001206963?sid=102
https://www.dnews.co.kr/
https://news.kbs.co.kr/common/main.html
https://n.news.naver.com/mnews/article/056/0011296271?sid=102
http://www.sportsseoul.com/
https://n.news.naver.com/mnews/article/468/0000862574?sid=101
https://www.dnews.co.kr/
https://www.mk.co.kr/
https://n.news.naver.com/mnews/article/009/0004986748?sid=102
http://www.munhwa.com/
https://n.news.naver.com/mnews/article/021/0002520878

In [37]:
###naver 기사 본문 및 제목 가져오기###

# ConnectionError방지
headers = { "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) Chrome/98.0.4758.102" }

titles = []
contents=[]
for i in naver_urls:
    original_html = requests.get(i,headers=headers)
    html = BeautifulSoup(original_html.text, "html.parser")
    # 검색결과확인시
    #print(html)
    
    #뉴스 제목 가져오기
    title = html.select("div#ct > div.media_end_head.go_trans > div.media_end_head_title > h2")
    # list합치기
    title = ''.join(str(title))
    # html태그제거
    pattern1 = '<[^>]*>'
    title = re.sub(pattern=pattern1,repl='', string=title)
    titles.append(title)

    #뉴스 본문 가져오기

    content = html.select("div#dic_area")

    # 기사 텍스트만 가져오기
    # list합치기
    content = ''.join(str(content))
    
    #html태그제거 및 텍스트 다듬기
    content = re.sub(pattern=pattern1,repl='', string=content)
    pattern2 = """[\n\n\n\n\n// flash 오류를 우회하기 위한 함수 추가\nfunction _flash_removeCallback() {}"""
    content = content.replace(pattern2,' ')

    contents.append(content)

print(titles)
print(contents)

['[소진공, 브릿지 리더십 위한 성장리더 워크숍 열어]', '[[뉴스킹] 강훈식 "평론가적 리더십으로는 당 이끌 수 없어"]', '[윤리위 D-1, 이준석 운명은…경고만 나와도 리더십 타격]', '[이상민, 윤희근 경찰청장 후보자 제청…"신망 두텁고 리더십 갖춰"]', "['전대 룰 뒤집기'에 상처 난 우상호 리더십…'친명계' 반발 일파만파]", '[이상민 장관 "윤희근 경찰청장 후보자, 리더십·신망 갖춰"]', '[이상민 장관 “경찰청장, 리더십 필요…일선 반발은 정치적”]', '[부드럽고 섬세한 리더십으로 친환경 뿌리산업 이끄는 여성 기업인 눈길]', "[경찰청 '한·아세안 경찰협력 리더십 포럼' 개최]", '[정치·경제위기 ‘대한민국 리빌딩’ 리더십 회복·통합 해법을 찾는다]', '[[영상] 지지율 하락...시험대 오른 윤석열 대통령 리더십]', '[[100대 CEO-신학철 LG화학 부회장]글로벌 리더십으로 저탄소 기반 신사업 육성]', '[이상민 장관 “경찰청장, 리더십 필요…일선 반발은 정치적”]', '[이재명 ‘강한 리더십’이냐, 97그룹 ‘세대교체론’이냐]', '[부드럽고 섬세한 리더십으로 친환경 뿌리산업 이끄는 여성 기업인 눈길]', '[[사설]尹 지지율 데드크로스… 위기 헤칠 리더십 보였나 성찰해야]', '[강병원 "의원들 의견 경청 않는 당대표, 리더십 쓸 수 있겠나"]', '[[사설] ‘前정권 비교’ 화법 접고 위기 극복 위한 ‘설득 리더십’ 보여라]', '[한-아세안 경찰협력 리더십 포럼 환영사하는 윤희근 경찰청차장]', '[스타벅스, 청년인재 8기 선발…장학금 및 리더십 양성 지원]', '[친문 홍영표 당대표 불출마 선언…새 리더십 강조하며 이재명 압박]', '[‘97그룹’ 강병원, 당대표 출마 “젊고 역동적 리더십으로 당 통합”]', '[]', '[[대청봉]지방자치 새 지평 열 리더십 요구]', '[인사 암초, 지지율 추락... 나토서 돌아온 尹, 리더십 시험대]', '[70년대생 강병원, 당대표 출마…"역동적 통합 리더십으로 일하겠

In [38]:
#데이터프레임으로 정리(titles,url,contents)
import pandas as pd

news_df = pd.DataFrame({'title':titles,'link':naver_urls,'content':contents})

news_df.to_csv('NaverNews_%s.csv'%search,index=False,encoding='utf-8-sig')